In [8]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.6 MB/s eta 0:00:00


In [1]:
import os
import pandas as pd
from PIL import Image
from torchvision import models, transforms
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from tqdm import tqdm
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import shutil
import timm
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q /content/drive/Shareddrives/CSE244A/combined.zip

In [4]:
# Custom dataset class
class ImageDataset(Dataset):
    def __init__(self, image_folder, csv_file, transform=None):
        self.image_folder = image_folder
        self.labels_df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.labels_df.iloc[idx, 0])
        label = int(self.labels_df.iloc[idx, 1])
        image = Image.open(img_name).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


# Load datasets
# df = pd.read_csv('./data-set/categories.csv')
# y_train = pd.read_csv('./data-set/train_labeled.csv')

# Paths
# labeled_image_folder = "./data-set/train/labeled"
# labeled_image_true_values = "./data-set/train_labeled.csv"

# unlabeled_image_folder = './data-set/train/unlabeled'
# combined_image_folder = './data-set/train/combined'

combined_csv_path = '/content/combined_labels.csv'


In [5]:
# Image transforms
transform_swin = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
combined_image_folder = '/content/combined'

In [7]:
# Create dataset and data loader
combined_dataset = ImageDataset(combined_image_folder, combined_csv_path, transform=transform_swin)

# Split the dataset into training (80%) and validation (20%)
train_size = int(0.8 * len(combined_dataset))
val_size = len(combined_dataset) - train_size
train_dataset, val_dataset = random_split(combined_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize your model
model = timm.create_model('swin_base_patch4_window7_224', pretrained=True)


# Freeze the feature extractors
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classification head
for param in model.head.parameters():
    param.requires_grad = True

# Unfreeze the nth block in the last stage
for param in model.layers[3].blocks[1].parameters():
    param.requires_grad = True

# Unfreeze the n-1th block in the last stage
for param in model.layers[3].blocks[0].parameters():
    param.requires_grad = True

# Update the classification head
num_classes = 135  # Example: Change to the number of classes in your dataset
model.head.fc = nn.Linear(model.head.fc.in_features, num_classes)

# Define optimizer and loss function
optimizer = torch.optim.AdamW([
    {'params': model.head.fc.parameters(), 'lr': 1e-3},  # Higher LR for the head
    {'params': model.layers[3].blocks[0].parameters(), 'lr': 1e-4},  # Lower LR for the last block
    {'params': model.layers[3].blocks[1].parameters(), 'lr': 5e-4},  # Lower LR for the last block
], weight_decay=1e-4)

criterion = nn.CrossEntropyLoss()

# Define learning rate scheduler based on training loss
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [8]:
!nvidia-smi

Sat Nov 30 03:44:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
import torch
print("\nPyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("GPU device name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())

# GPU Memory management functions
def get_gpu_memory():
    """Print GPU memory usage"""
    if torch.cuda.is_available():
        print(f"Allocated: {torch.cuda.memory_allocated(0)/1024**2:.2f}MB")
        print(f"Cached: {torch.cuda.memory_reserved(0)/1024**2:.2f}MB")

def clear_gpu_memory():
    """Clear GPU memory"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

# Device selection function
def get_device():
    """Get appropriate device"""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    return device

# Get the device
device = get_device()


PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
GPU device name: Tesla T4
Number of GPUs: 1
Using GPU: Tesla T4


In [10]:
# Move model to GPU if available
model = model.to(device)

In [ ]:
# Initialize lists to store validation loss and accuracy
val_losses = []
val_accuracies = []

# Training loop with validation
epochs = 5  # Adjust the number of epochs as needed
for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        # logits = outputs.logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss / len(train_loader):.4f}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # logits = outputs.logits
            loss = criterion(outputs, labels)  # Calculate validation loss
            val_loss += loss.item()  # Accumulate validation loss
            _, predicted = torch.max(outputs, 1)  # Get predicted classes
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    val_losses.append(val_loss / len(val_loader))  # Store average validation loss
    val_accuracies.append(accuracy * 100)  # Store validation accuracy in percentage

    print(f'Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {accuracy * 100:.2f}%')

Epoch 1/5:  89%|████████▉ | 690/776 [06:33<00:52,  1.63it/s]

In [ ]:
# Plotting validation loss and accuracy
plt.figure(figsize=(12, 5))

# Plot validation loss
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), val_losses, marker='o', label='Validation Loss', color='blue')
plt.title('Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(range(1, epochs + 1))
plt.grid()
plt.legend()

# Plot validation accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), val_accuracies, marker='o', label='Validation Accuracy', color='green')
plt.title('Validation Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.xticks(range(1, epochs + 1))
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()